<center>

# 🧠 System & Environment Inspector 🚀  
### **Comprehensive Report for AI Workflows**

</center>

---

This notebook gathers detailed information about the **system environment**, including:  
- 💻 **CPU, RAM, and GPU** specifications  
- ⚙️ **Operating System** and **Python** version  
- 🔍 **CUDA availability** and installed packages like **PyTorch** & **TensorFlow**  
- 🌐 **Network** and IP details  

Use this notebook at the start of any project to ensure your **hardware and software environment** are ready for AI tasks.

### Install required dependencies

In [24]:
%pip install py-cpuinfo
%pip install tensorflow
%pip install pynvml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/53.1 kB ? eta -:--:--
   ---------------------------------------- 53.1/53.1 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
import platform

def check_cpuinfo_installed() -> None:
    if platform.system() == "Windows":
        #os.system("pip list | FindStr py-cpuinfo")
        !pip list | FindStr "py-cpuinfo tensorflow nvidia-ml-py"
    else:
        #os.system("pip list | grep py-cpuinfo")
        !pip list | grep "py-cpuinfo tensorflow nvidia-ml-py"

check_cpuinfo_installed()

nvidia-ml-py                 12.560.30
py-cpuinfo                   9.0.0
tensorflow                   2.17.0
tensorflow-intel             2.17.0
tensorflow-io-gcs-filesystem 0.31.0


In [2]:
import psutil
import cpuinfo
import torch
import pynvml
import socket
import platform
import subprocess
import tensorflow as tf

from typing import Any, TextIO

def get_system_info() -> dict[str, Any]:
    return {
        "Platform": f"{platform.system()} {platform.release()}",
        "Machine": platform.machine(),
        "Processor": cpuinfo.get_cpu_info()['brand_raw']
    }

def get_ram_info() -> dict[str, str]:
    ram = psutil.virtual_memory()
    total_ram = ram.total / (1024 ** 3)
    available_ram = ram.available / (1024 ** 3)
    used_ram = total_ram - available_ram
    return {
        "Total RAM": f"{total_ram:.2f} GB",
        "Available RAM": f"{available_ram:.2f} GB",
        "Used RAM": f"{used_ram:.2f} GB",
        "Memory Usage": f"{ram.percent}%"
    }

def get_cpu_info()-> dict[str, Any]:
    return {
        "CPU Cores (Logical)": psutil.cpu_count(logical=True),
        "CPU Cores (Physical)": psutil.cpu_count(logical=False),
        "CPU Frequency": f"{psutil.cpu_freq().max:.2f} MHz",
        "CPU Usage": f"{psutil.cpu_percent(interval=1)}%"
    }

def get_gpu_info() -> dict[str, Any]:
    if torch.cuda.is_available():
        try:
            pynvml.nvmlInit()
            handle = pynvml.nvmlDeviceGetHandleByIndex(0)
            info = pynvml.nvmlDeviceGetMemoryInfo(handle)
            
            total_memory = info.total / (1024 ** 3)
            used_memory = info.used / (1024 ** 3)
            free_memory = info.free / (1024 ** 3)
            percent_used = (used_memory / total_memory) * 100
            
            # Get additional information
            gpu_name = pynvml.nvmlDeviceGetName(handle)
            driver_version = pynvml.nvmlSystemGetDriverVersion()
            
            return {
                "CUDA Available": "Yes",
                "GPU Name": gpu_name,
                "CUDA Version": torch.version.cuda,
                "Driver Version": driver_version,
                "Total GPU Memory": f"{total_memory:.2f} GB",
                "Used GPU Memory": f"{used_memory:.2f} GB",
                "Free GPU Memory": f"{free_memory:.2f} GB",
                "Memory Usage": f"{percent_used:.2f}%"
            }
        except pynvml.NVMLError as error:
            return {"Error": f"NVML Error: {error}"}
        finally:
            pynvml.nvmlShutdown()
    return {"CUDA Available": "No"}

def get_tensorflow_gpu_info() -> dict[str, bool]:
    return {"TensorFlow GPU Available": tf.config.list_physical_devices('GPU') != []}

def get_os_specific_info() -> dict: 
    os_info = {}
    if platform.system() == "Linux":
        try:
            distro = " ".join(platform.linux_distribution())
        except AttributeError:
            distro = " ".join(platform.freedesktop_os_release().values())
        os_info["Linux Distro"] = distro
        os_info["Kernel Version"] = platform.release()
    elif platform.system() == "Windows":
        os_info["Windows Version"] = platform.version()
    elif platform.system() == "Darwin":
        os_info["MacOS Version"] = platform.mac_ver()[0]
    return os_info

def get_installed_packages() -> dict[str, str]:
    packages = {
        "Python Version": platform.python_version()
    }
    try:
        packages["PyTorch Version"] = torch.__version__
    except:
        packages["PyTorch"] = "not installed"
    
    try:
        packages["TensorFlow Version"] = tf.__version__
    except:
        packages["TensorFlow"] = "not installed"
    
    return packages

def get_network_info() -> dict[str, str]:
    try:
        hostname = platform.node()
        if platform.system() == "Windows":
            local_ip = socket.gethostbyname(hostname)
        else:  # Linux/MacOS
            local_ip = subprocess.check_output(['hostname', '-I']).decode().strip().split()[0]
        return {
            "Hostname": hostname,
            "Local IP": local_ip
        }
    except Exception as e:
        return {"Error": f"Unable to retrieve network information: {e}"}

def print_section(title: str, data: dict) -> None:
    print(f"\n=== {title} ===")
    for key, value in data.items():
        print(f"{key}: {value}")

def write_section(file: TextIO, title: str, data: dict) -> None:
    file.write(f"\n=== {title} ===\n")
    for key, value in data.items():
        file.write(f"{key}: {value}\n")

def environment_info() -> None:
    sections = [
        ("SYSTEM INFORMATION", get_system_info()),
        ("RAM INFORMATION", get_ram_info()),
        ("CPU INFORMATION", get_cpu_info()),
        ("GPU INFORMATION", get_gpu_info()),
        ("TENSORFLOW GPU INFORMATION", get_tensorflow_gpu_info()),
        ("OS-SPECIFIC INFORMATION", get_os_specific_info()),
        ("INSTALLED PACKAGE VERSIONS", get_installed_packages()),
        ("NETWORK INFORMATION", get_network_info())
    ]

    # Print to console
    for title, data in sections:
        print_section(title, data)

    # Write to file
    with open('output.txt', 'w') as f:
        for title, data in sections:
            write_section(f, title, data)

    print("\nEnvironment information has been written to output.txt")

if __name__ == "__main__":
    environment_info()


=== SYSTEM INFORMATION ===
Platform: Windows 10
Machine: AMD64
Processor: 13th Gen Intel(R) Core(TM) i9-13900HX

=== RAM INFORMATION ===
Total RAM: 31.75 GB
Available RAM: 13.16 GB
Used RAM: 18.58 GB
Memory Usage: 58.5%

=== CPU INFORMATION ===
CPU Cores (Logical): 32
CPU Cores (Physical): 24
CPU Frequency: 2200.00 MHz
CPU Usage: 1.5%

=== GPU INFORMATION ===
CUDA Available: Yes
GPU Name: NVIDIA GeForce RTX 4080 Laptop GPU
CUDA Version: 11.8
Driver Version: 560.94
Total GPU Memory: 11.99 GB
Used GPU Memory: 2.13 GB
Free GPU Memory: 9.87 GB
Memory Usage: 17.72%

=== TENSORFLOW GPU INFORMATION ===
TensorFlow GPU Available: False

=== OS-SPECIFIC INFORMATION ===
Windows Version: 10.0.22631

=== INSTALLED PACKAGE VERSIONS ===
Python Version: 3.9.18
PyTorch Version: 2.2.1+cu118
TensorFlow Version: 2.17.0

=== NETWORK INFORMATION ===
Hostname: LAPTOP-H9HLH4K2
Local IP: 192.168.150.1

Environment information has been written to output.txt
